In [1]:
import tensorflow as tf
print("imported tensorflow")
import os
print("imported os")
import matplotlib
from matplotlib import pyplot as plt
import cv2
import imghdr
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

imported tensorflow
imported os


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
gpus

for gpu in gpus:
    gpu = tf.config.experimental.set_memory_growth(gpu,True)

In [3]:
data = 'Dataset'
os.listdir(data)

image_exts = ['jpeg', 'jpg', 'bmp', 'png','JPEG','PNG','BMP'] 

In [4]:
for rock_type in os.listdir(data):
    rock_type_path = os.path.join(data, rock_type)
    
    # Skip if not a directory
    if not os.path.isdir(rock_type_path):
        continue
    
    # Loop through actual images in each rock type folder
    for image in os.listdir(rock_type_path):
        image_path = os.path.join(rock_type_path, image)
        
        # Skip if it's a directory (shouldn't be any)
        if os.path.isdir(image_path):
            continue
        
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {} - {}'.format(image_path, e))
            
print("Issues resolved")

Issues resolved


In [5]:
data = tf.keras.utils.image_dataset_from_directory('Dataset', shuffle = True) #ts is a pipeline
class_names = data.class_names

Found 1988 files belonging to 7 classes.


In [6]:
print(class_names)
#0-6

['Basalt', 'Coal', 'Granite', 'Limestone', 'Marble', 'Quartzite', 'Sandstone']


In [7]:
#normalizing the data...
data = data.map(lambda x,y: (x/255, y))

In [8]:
len(data)

63

In [ ]:
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.15)
test_size = len(data) - train_size - val_size

print("Batches ->", train_size, val_size, test_size)
print("Total batches:", train_size + val_size + test_size)

In [10]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [11]:
sample_batch = next(iter(train))
print("Min pixel value:", tf.reduce_min(sample_batch[0]).numpy())
print("Max pixel value:", tf.reduce_max(sample_batch[0]).numpy())
print("Sample pixel:", sample_batch[0][0][100][100].numpy())
print("Data type:", sample_batch[0].dtype)
print(f"Training batch labels: {sample_batch[1].numpy()}")
print(f"Unique labels: {set(sample_batch[1].numpy())}")
print(f"Class names: {class_names}")

Min pixel value: 0.0
Max pixel value: 1.0
Sample pixel: [0.38972756 0.30792776 0.22391543]
Data type: <dtype: 'float32'>
Training batch labels: [4 4 3 5 4 1 3 1 5 0 6 5 3 2 3 1 4 3 1 4 5 1 4 1 5 4 0 6 5 5 6 5]
Unique labels: {0, 1, 2, 3, 4, 5, 6}
Class names: ['Basalt', 'Coal', 'Granite', 'Limestone', 'Marble', 'Quartzite', 'Sandstone']
